In [1]:
%load_ext autoreload
%autoreload 2
!pwd


/Users/eric/proj/league-projects/leaguesync/notebooks


In [2]:
#!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib


In [3]:
import datetime
import os.path

from leaguesync.util import get_config

from leaguesync.calendar import * 

from leaguesync import * 
from leaguesync.calendar import logger
from leaguesync.calendar import Calendar
import logging

logging.basicConfig()
logger.setLevel(logging.DEBUG)

from tqdm.auto import tqdm

import warnings

p13 = Pike13('./league-remote.env')

pdf = Pike13DataFrames(p13)

config = get_config('./league-remote.env')
cv_cal_id = config['GA_CLASSES_CAL_CV']
ncv_cal_id = config['GA_CLASSES_CAL_NOTCV']
vl_cal_id=config['GA_CLASSES_CAL_VL']


In [4]:
cal = Calendar('./league-remote.env', location="CV", cal_id=cv_cal_id)
cal.update(p13)

DEBUG:leaguesync.calendar:Got 791 events from Pike13
DEBUG:leaguesync.calendar:Got 791 events from Google Calendar
DEBUG:leaguesync.calendar:Got 791 events in both Pike13 and Google Calendar
DEBUG:leaguesync.calendar:Got 0 events in Pike13 only
DEBUG:leaguesync.calendar:Got 0 events in Google Calendar only


In [5]:
def event_f(e):
    """Return true if this event should be included in the non-CV calendar, which is actually
    not CV, and not Virtual"""

    return Calendar.is_location(e, '!CV') and Calendar.is_location(e, '!VL')

ncal = Calendar('./league-remote.env', event_f=event_f, cal_id=ncv_cal_id)
ncal.update()


DEBUG:leaguesync.calendar:Got 35 events from Pike13
DEBUG:leaguesync.calendar:Got 35 events from Google Calendar
DEBUG:leaguesync.calendar:Got 35 events in both Pike13 and Google Calendar
DEBUG:leaguesync.calendar:Got 0 events in Pike13 only
DEBUG:leaguesync.calendar:Got 0 events in Google Calendar only


In [6]:
vlcal = Calendar('./league-remote.env', location="VL", cal_id=vl_cal_id)
vlcal.update(p13)

DEBUG:leaguesync.calendar:Got 558 events from Pike13
DEBUG:leaguesync.calendar:Got 558 events from Google Calendar
DEBUG:leaguesync.calendar:Got 558 events in both Pike13 and Google Calendar
DEBUG:leaguesync.calendar:Got 0 events in Pike13 only
DEBUG:leaguesync.calendar:Got 0 events in Google Calendar only


In [7]:
ncal = Calendar('./league-remote.env', cal_id=ncv_cal_id)
#ncal.delete_events(progress=True)

In [8]:
ncal.update_events()

DEBUG:leaguesync.calendar:Got 1384 events from Pike13
DEBUG:leaguesync.calendar:Got 39 events from Google Calendar


In [52]:
cale = ncal.pike13_events_df()
cale.head()

,event_occurrence_id,event_id,event_name,service_id,location_id,state,visits_count,start_at,end_at,occ_month,...,location_code,service_name,type,duration_in_minutes,maximum_clients,category_id,category_name,description,description_short,instructions
20764,193284153,7428205,Java@VL Panda,191042,23281,canceled,1,2024-01-01 17:00:00-08:00,2024-01-01 18:30:00-08:00,2024-01,...,VL,Java@VL,GroupClass,90,5,88168.0,All Virtual Classes,None,None,None
20765,193284443,8792650,Java@CV Finch,263387,22675,canceled,2,2024-01-01 17:00:00-08:00,2024-01-01 18:30:00-08:00,2024-01,...,CV,Java@CV,GroupClass,90,6,88168.0,All Virtual Classes,None,None,None
20766,193287757,7469782,Java@VL Sea Bass,191042,23281,canceled,2,2024-01-01 17:00:00-08:00,2024-01-01 18:30:00-08:00,2024-01,...,VL,Java@VL,GroupClass,90,5,88168.0,All Virtual Classes,None,None,None
20767,193284947,9912365,Make-Up Lab @VL Java/Python (small),266276,23281,canceled,0,2024-01-01 18:30:00-08:00,2024-01-01 20:00:00-08:00,2024-01,...,VL,Make-Up Lab @VL Java/Python (small),GroupClass,90,2,88168.0,All Virtual Classes,Make-Up Lab is available for League students e...,None,None
20768,193286627,5842451,Java@VL Dolphin,191042,23281,canceled,0,2024-01-01 18:30:00-08:00,2024-01-01 20:00:00-08:00,2024-01,...,VL,Java@VL,GroupClass,90,5,88168.0,All Virtual Classes,None,None,None


In [30]:
Calendar.filter_location(cale,['!VL', '!CV']).location_name.value_counts()

location_name
Hoover High School (HHS)    22
LINCOLN HIGH SCHOOL         13
Name: count, dtype: int64

In [ ]:
for e, prv in cale:
    print(e['summary'], e['start'], e['description'])

In [38]:
cale.columns

Index(['event_occurrence_id', 'event_id', 'event_name', 'service_id',
       'location_id', 'state', 'visits_count', 'start_at', 'end_at',
       'occ_month', 'occ_week', 'dow', 'location_name', 'latitude',
       'longitude', 'location_code', 'service_name', 'type',
       'duration_in_minutes', 'maximum_clients', 'category_id',
       'category_name', 'description', 'description_short', 'instructions'],
      dtype='object')

In [108]:
def in_date_range(df):
    """Filter the DataFrame to only include events in the current week and the next 30 days"""

    from datetime import datetime, timedelta

    start_date = datetime.now().date() - timedelta(days=datetime.now().weekday())
    end_date = datetime.now().date() + timedelta(days=30)

    mask = (df['start_at'] >= str(start_date)) & (df['start_at'] <= str(end_date) )
    return df.loc[mask]
    
def pike13_events_df(p13: Pike13, location=None, select_f=None, incl_str=None):
    """Get the events from Pike13 as a DataFrame, form on month ago to now."""

    pdf = Pike13DataFrames(p13)

    eo = pdf.event_occurrences
    loc = pdf.locations
    srv = pdf.services

    t = eo.merge(loc).merge(srv)
    t = t[t.start_at >= str(one_month_ago().date())]

    if location:
        t = Calendar.filter_location(t, location)

    if select_f:
        t = t[t.apply(select_f, axis=1)]

    if includes_str:
        t = includes_str(t, incl_str)

    t = in_date_range(t)

    # Ensure the datetime is timezone-aware, set to UTC
    for c in ['start_at', 'end_at']:
        t[c] = t[c].apply(lambda x: convert_naive(x).tz_convert('America/Los_Angeles'))
        
    return t

In [146]:
df[['event_id','service_id']].drop_duplicates()

,event_id,service_id
21007,9804197,191042
21008,8792522,263387
21009,13293124,213851
21010,7428205,191042
21011,7469782,191042
...,...,...
21099,13396298,292457
21100,13396344,257950
21107,10801766,292822
21175,13378382,319359


In [147]:
%run /Users/eric/proj/league-projects/website-services/util.py 

def select(event):
    return 'java' in event.event_name.lower() and not 'make-up' in event.event_name.lower()

df = pike13_events_df(p13)

# Add the day of week and hours strings, such as
# Sundays at 4:00PM, Saturdays at 4:00PM

t = make_when_df(df)
t.head()      


,event_id,event_name,location_name,description_short,description,dow_hours,sort_order,dh_hour,dh_hour_str,dh_dow,dh_dow_str,when,service_id
0,13407638,Programming Classes,Carmel Valley Campus (CV),"Currently the worlds most popular language, Py...",None,"{'hour': 11, 'dow': 0}",11,11,11:00AM,0,Mondays,Mondays at 11:00AM,319636
1,13407638,Programming Classes,Carmel Valley Campus (CV),"Currently the worlds most popular language, Py...",None,"{'hour': 11, 'dow': 2}",11,11,11:00AM,2,Wednesdays,Wednesdays at 11:00AM,319636
2,9804197,Java@VL Starling,Virtual Learning (VL),None,None,"{'hour': 15, 'dow': 0}",15,15,3:00PM,0,Mondays,Mondays at 3:00PM,191042
3,8792522,Java@CV Cobra,Carmel Valley Campus (CV),None,None,"{'hour': 15, 'dow': 0}",15,15,3:00PM,0,Mondays,Mondays at 3:00PM,263387
4,13293124,Java@LH,LINCOLN HIGH SCHOOL,None,None,"{'hour': 15, 'dow': 0}",15,15,3:00PM,0,Mondays,Mondays at 3:00PM,213851


In [107]:
t =df.merge(events[['event_id','dow_hours','dh_str']])
t[['dow_hours','dh_str','start_at

,event_occurrence_id,event_id,event_name,service_id,location_id,state,visits_count,start_at,end_at,occ_month,...,type,duration_in_minutes,maximum_clients,category_id,category_name,description,description_short,instructions,dow_hours,dh_str
0,195738447,9804197,Java@VL Starling,191042,23281,active,1,2024-01-29 23:30:00+00:00,2024-01-30 01:00:00+00:00,2024-01,...,GroupClass,90,5,88168.0,All Virtual Classes,None,None,None,"[{'hour': 23, 'dow': 0}]",Mondays at 11:00PM
1,195741429,8792522,Java@CV Cobra,263387,22675,active,5,2024-01-29 23:30:00+00:00,2024-01-30 01:00:00+00:00,2024-01,...,GroupClass,90,6,88168.0,All Virtual Classes,None,None,None,"[{'hour': 23, 'dow': 0}]",Mondays at 11:00PM
2,223311271,13293124,Java@LH,213851,39107,active,0,2024-01-29 23:30:00+00:00,2024-01-30 01:00:00+00:00,2024-01,...,GroupClass,90,10,88168.0,All Virtual Classes,None,None,None,"[{'hour': 23, 'dow': 0}]",Mondays at 11:00PM
3,195735867,7428205,Java@VL Panda,191042,23281,active,1,2024-01-30 01:00:00+00:00,2024-01-30 02:30:00+00:00,2024-01,...,GroupClass,90,5,88168.0,All Virtual Classes,None,None,None,"[{'hour': 1, 'dow': 1}]",Tuesdays at 1:00AM
4,195738644,7469782,Java@VL Sea Bass,191042,23281,active,3,2024-01-30 01:00:00+00:00,2024-01-30 02:30:00+00:00,2024-01,...,GroupClass,90,5,88168.0,All Virtual Classes,None,None,None,"[{'hour': 1, 'dow': 1}]",Tuesdays at 1:00AM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262,198649287,6768695,Java@CV Bumble Bee,191042,22675,active,1,2024-03-03 19:30:00+00:00,2024-03-03 21:00:00+00:00,2024-03,...,GroupClass,90,5,88168.0,All Virtual Classes,None,None,None,"[{'hour': 19, 'dow': 6}]",Sundays at 7:00PM
263,198650824,5345605,Java@VL Crocodile,191042,23281,active,3,2024-03-03 19:30:00+00:00,2024-03-03 21:00:00+00:00,2024-03,...,GroupClass,90,5,88168.0,All Virtual Classes,None,None,None,"[{'hour': 19, 'dow': 6}]",Sundays at 7:00PM
264,198650262,8942020,Java@CV Penguin,263387,22675,active,3,2024-03-03 21:00:00+00:00,2024-03-03 22:30:00+00:00,2024-03,...,GroupClass,90,6,88168.0,All Virtual Classes,None,None,None,"[{'hour': 21, 'dow': 6}]",Sundays at 9:00PM
265,203277474,11935302,Java@VL Seal,191042,23281,active,1,2024-03-03 21:00:00+00:00,2024-03-03 22:30:00+00:00,2024-03,...,GroupClass,90,5,88168.0,All Virtual Classes,None,None,None,"[{'hour': 21, 'dow': 6}]",Sundays at 9:00PM


In [163]:
from leaguesync.web.util import render
from leaguesync.web import render_events_html

def select(event):
    return (
            'java' in event.event_name.lower() and
            not 'make-up' in event.event_name.lower() and
            not '@' in event.event_name.lower() and
            event.location_code == 'CV'
            )

from IPython.display import HTML
HTML(render_events_html(p13, select_f=select))
